# TinyML ECG Classification

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

## Read Dataset

In [ ]:
df_train = pd.read_csv("/kaggle/input/heartbeat/mitbih_train.csv", header=None)
df_test = pd.read_csv("/kaggle/input/heartbeat/mitbih_test.csv", header=None)
df_train.head()

In [ ]:
# df[187] is y (class 0~4)
print(df_train[187].value_counts())
print(df_test[187].value_counts())

## Prepare Dataset

In [ ]:
X_train = df_train.values[:, :-1]
y_train = df_train.values[:, -1].astype(int)

X_test  = df_test.values[:, :-1]
y_test  = df_test.values[:, -1].astype(int)

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1],1)

print(X_train.shape)
print(X_test.shape)

In [ ]:
from tensorflow.keras import utils
y_train = utils.to_categorical(y_train)
y_test  = utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

## Build Model

In [ ]:
input_shape = X_train.shape[1:]
num_classes= 5

### *Conv1D not supported by TFLite for Micro, so Conv2D is used, not much difference in code size*

In [ ]:
# Build Model 
from tensorflow.keras import models, layers

inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(16, kernel_size=5, activation='relu', padding='same')(inputs)
#x = layers.Conv2D(16, kernel_size=5, activation='relu', padding='same')(x)
#x = layers.MaxPooling2D((2,2), padding='same')(x)

x = layers.Flatten()(x)
x = layers.Dense(16)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = models.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
# Compile Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train Model

In [ ]:
batch_size = 100
num_epochs = 10 

In [ ]:
# Train Model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test))

## Save Model

In [ ]:
# Save Model
models.save_model(model, 'ecg_arrhythmia.hdf5')

## TinyML

In [ ]:
!pip install tinymlgen

### export model
    import tensorflow as tf
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()

    # Save the model to disk
    open("ecg_arrhythmia.tflite", "wb").write(tflite_model)
    
    # hex dump
    !xxd -i ecg_arrhythmia.tflite > ecg_arrhythmia.cc

In [ ]:
from tinymlgen import port
c_code = port(model, variable_name='ecg_arrhythmia', pretty_print=True, optimize=False) 

with open('ecg_arrhythmia.h', 'w') as f:
    print(c_code, file=f)

### export a test data

In [ ]:
with open('x_test.h', 'w') as f:
    print("float x_test_dat[187] = {", file=f)    
    s="    "
    for i in range(187):
        s+=str(X_test[0][0][i][0])+', '
        if i==186:
            print(s, file=f) 
        elif (i+1)%8==0:
            print(s, file=f)
            s="    "
    print("};", file=f)
f.close()

In [ ]:
!cat x_test.h

In [ ]:
print(y_test[0])

In [ ]:
plt.plot(X_test[0][0])